In [9]:
from isochrones.dartmouth import Dartmouth_Isochrone
dar = Dartmouth_Isochrone()

3 stars physically bound wih mass 0.5Msun, 1.0Msun , 2.0Msun

In [11]:
from isochrones.utils import addmags
import numpy as np
M1 = 2.
M2 = 1.0
M3 = 0.5
age = np.log10(5e8)
feh = 0.0
distance = 500
AV = 0.1

unresolved_bands = ['i','J','H','K','W1','W2']
resolved_bands = ['i','J','K']
args = (age, feh, distance, AV)
unresolved = {b:addmags(dar.mag[b](M1, *args), dar.mag[b](M2, *args), dar.mag[b](M3, *args)) for b in unresolved_bands}
resolved_1 = {b:dar.mag[b](M1, *args) for b in resolved_bands}
resolved_2 = {b:dar.mag[b](M2, *args) for b in resolved_bands}
resolved_3 = {b:dar.mag[b](M3, *args) for b in resolved_bands}

ValueError: number of dimensions in xi does not match x

In [4]:
unresolved, resolved_1, resolved_2, resolved_3

NameError: name 'unresolved' is not defined

In [5]:
import pandas as pd

instruments = ['twomass','WISE','SDSS','RAO']
bands = {'twomass':['J','H','K'],
          'WISE':['W1','W2'],
          'SDSS':['i'],
          'RAO':['i','J','K']}
mag_unc = {'twomass': 0.02, 'WISE':0.02, 'SDSS':0.02, 'RAO':0.1}
resolution = {'twomass':4.0, 'WISE':5.0, 'SDSS':4.5, 'RAO':0.1}
relative = {'twomass':False, 'WISE':False, 'SDSS':False, 'RAO':True}
separation2 = 0.5
separation3 = 1.2
PA2, PA3 = 100.,45.

columns = ['name', 'band', 'resolution', 'relative', 'separation', 'pa', 'mag', 'e_mag']
df = pd.DataFrame(columns=columns)
i=0
for inst in ['twomass', 'WISE','SDSS']:  #Unresolved observations
    for b in bands[inst]:
        row = {}
        row['name'] = inst
        row['band'] = b
        row['resolution'] = resolution[inst]
        row['relative'] = relative[inst]
        row['separation'] = 0.
        row['pa'] = 0.
        row['mag'] = unresolved[b]
        row['e_mag'] = mag_unc[inst]
        df = df.append(pd.DataFrame(row, index=[i]))
        i += 1

for inst in ['RAO']:  #Resolved observations
    for b in bands[inst]:
        mags = [resolved_1[b], resolved_2[b], resolved_3[b]]
        pas = [0, PA2, PA3]
        seps = [0., separation2, separation3]
        for mag,sep,pa in zip(mags,seps,pas):
            row = {}
            row['name'] = inst
            row['band'] = b
            row['resolution'] = resolution[inst]
            row['relative'] = relative[inst]
            row['separation'] = sep
            row['pa'] = pa
            row['mag'] = mag
            row['e_mag'] = mag_unc[inst]
            df = df.append(pd.DataFrame(row, index=[i]))
            i += 1
            
df

NameError: name 'unresolved' is not defined

In [25]:
from isochrones.observation import ObservationTree

In [28]:
t = ObservationTree.from_df(df, name='test-triplet')
t.print_ascii()

test-triplet
 ╚═ WISE W1=(9.69, 0.02) @(0.00, 0 [5.00])
    ╚═ WISE W2=(9.69, 0.02) @(0.00, 0 [5.00])
       ╚═ SDSS i=(10.25, 0.02) @(0.00, 0 [4.50])
          ╚═ twomass H=(9.73, 0.02) @(0.00, 0 [4.00])
             ╚═ twomass J=(9.78, 0.02) @(0.00, 0 [4.00])
                ╚═ twomass K=(9.70, 0.02) @(0.00, 0 [4.00])
                   ╠═ RAO J=(9.88, 0.10) @(0.00, 0 [0.10])
                   ║  ╚═ RAO K=(9.85, 0.10) @(0.00, 0 [0.10])
                   ║     ╚═ RAO i=(10.32, 0.10) @(0.00, 0 [0.10])
                   ╠═ RAO J=(12.46, 0.10) @(0.50, 100 [0.10])
                   ║  ╚═ RAO K=(12.07, 0.10) @(0.50, 100 [0.10])
                   ║     ╚═ RAO i=(13.38, 0.10) @(0.50, 100 [0.10])
                   ╚═ RAO J=(15.16, 0.10) @(1.20, 45 [0.10])
                      ╚═ RAO K=(14.32, 0.10) @(1.20, 45 [0.10])
                         ╚═ RAO i=(17.00, 0.10) @(1.20, 45 [0.10])


In [8]:
t.define_models(dar)
t.print_ascii()

NameError: name 't' is not defined

In [12]:
t.add_limit(logg=(3.0,None))
t.print_ascii()

NameError: name 't' is not defined

In [13]:
from isochrones.starmodel import StarModel
mod = StarModel(dar, obs=t)

NameError: name 't' is not defined

In [14]:
mod.fit_multinest(n_live_points=1000)

NameError: name 'mod' is not defined

In [15]:
mod.samples[['mass_0_0','mass_0_1','mass_0_2','age_0','feh_0','distance_0','AV_0']].head()

NameError: name 'mod' is not defined

In [16]:
%matplotlib inline
#mod.corner(['mass_0_0', 'mass_0_1','logg_0_0','age_0','distance_0','AV_0']);
mod.corner_physical();

NameError: name 'mod' is not defined

In [17]:
mod.corner_observed();

NameError: name 'mod' is not defined

In [18]:
mod.obs.print_ascii(p=[M1,M2,age,feh,distance,AV])

NameError: name 'mod' is not defined

In [19]:
t2 = ObservationTree.from_df(df, name='triple2')
t2.define_models(dar, index=[0,1,2])
t2.print_ascii()

NameError: name 'ObservationTree' is not defined

In [20]:
mod2 = StarModel(dar, obs=t2)
mod2.fit_multinest(basename='unassoc')

NameError: name 't2' is not defined

In [21]:
mod2.corner_physical();

NameError: name 'mod2' is not defined

In [22]:
mod2.corner_observed();

NameError: name 'mod2' is not defined

In [23]:
mod.evidence

NameError: name 'mod' is not defined

In [24]:
mod2.evidence

NameError: name 'mod2' is not defined